In [1]:
import rects
# import the necessary packages
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread
import numpy as np
import playsound
import argparse
import imutils
import time
import dlib
import cv2

In [2]:
def sound_alarm(path):
	# play an alarm sound
	playsound.playsound(path)

In [3]:
def eye_aspect_ratio(eye):
	# compute the euclidean distances between the two sets of
	# vertical eye landmarks (x, y)-coordinates
	A = dist.euclidean(eye[1], eye[5])
	B = dist.euclidean(eye[2], eye[4])
	# compute the euclidean distance between the horizontal
	# eye landmark (x, y)-coordinates
	C = dist.euclidean(eye[0], eye[3])
	# compute the eye aspect ratio
	ear = (A + B) / (2.0 * C)
	# return the eye aspect ratio
	return ear

In [4]:
import sys
sys.argv = ['your_script.py', '-p', 'path/to/shape_predictor_68_face_landmarks.dat']

# Argument parsing
ap = argparse.ArgumentParser()
ap.add_argument("-p", "--shape-predictor", required=True, help="shape_predictor_68_face_landmarks.dat")
ap.add_argument("-a", "--alarm", type=str, default="384967__bajko__sfx_seatbelt-warning_auto.wav", help="Path to alarm .WAV file")
ap.add_argument("-w", "--webcam", type=int, default=0, help="index of webcam on system")
args = vars(ap.parse_args())

print(args)



{'shape_predictor': 'path/to/shape_predictor_68_face_landmarks.dat', 'alarm': '384967__bajko__sfx_seatbelt-warning_auto.wav', 'webcam': 0}


In [5]:
# define two constants, one for the eye aspect ratio to indicate
# blink and then a second constant for the number of consecutive
# frames the eye must be below the threshold for to set off the
# alarm
EYE_AR_THRESH = 0.2
EYE_AR_CONSEC_FRAMES = 48
# initialize the frame counter as well as a boolean used to
# indicate if the alarm is going off
COUNTER = 0
ALARM_ON = False

In [6]:
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
print("[INFO] loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

[INFO] loading facial landmark predictor...


In [7]:
# grab the indexes of the facial landmarks for the left and
# right eye, respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [8]:
import cv2
from imutils.video import VideoStream
import time
from threading import Thread
import imutils

# Global alarm state
ALARM_ON = False

def sound_alarm(file_path):
    # Play the alarm sound (implement this function as needed)
    # You can use libraries such as playsound to play the .wav file
    from playsound import playsound
    playsound(file_path)

# Start the video stream thread
print("[INFO] starting video stream thread...")
vs = VideoStream(src=args["webcam"]).start()  # Start video stream
time.sleep(1.0)

try:
    # Loop over frames from the video stream
    while True:
        # Grab the frame from the threaded video file stream, resize it, and convert to grayscale
        frame = vs.read()
        frame = imutils.resize(frame, width=450)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the grayscale frame
        rects = detector(gray, 0)

        # Loop over the face detections
        for rect in rects:
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)
            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0

            # Compute the convex hull for the left and right eye
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

            # Check if the eye aspect ratio is below the threshold
            if ear < EYE_AR_THRESH:
                COUNTER += 1
                if COUNTER >= EYE_AR_CONSEC_FRAMES:
                    if not ALARM_ON:
                        ALARM_ON = True
                        if args["alarm"] != "":
                            t = Thread(target=sound_alarm, args=(args["alarm"],))
                            t.daemon = True
                            t.start()
                    cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                COUNTER = 0
                ALARM_ON = False

            cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Show the frame
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF

        # If the `q` key was pressed, break from the loop
        if key == ord("q"):
            break

except Exception as e:
    print(f"[ERROR] {e}")

finally:
    print("[INFO] Cleaning up...")

    # Turn off alarm if it's playing
    if ALARM_ON:
        print("[INFO] Stopping the alarm...")
        ALARM_ON = False

    # Properly stop the video stream and release webcam resources
    if vs is not None:
        vs.stop()
    if hasattr(vs, "stream"):
        vs.stream.release()

    # Close all OpenCV windows
    cv2.destroyAllWindows()
    print("[INFO] Webcam and video stream stopped.")


[INFO] starting video stream thread...
[INFO] Cleaning up...
[INFO] Webcam and video stream stopped.
